<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/2020_02_CNN_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- in images, let us assume that we want to classify the image 
    - in early labs of deep learning, we used Dense layers to do that 
        - flatten the 2D image to a very long vector 
        - input it to the NN 
        - but we said that this procedure has a lot of weights that need to be trained 
- then the idea of convolution from image processing got introduced into deep learning , remeber in image processing we used to have certain filters that do certain operations (detecting horiontal edges, vertical edges, sharpining and so on) and convolute the filters on the image 

- so people thought about making the NN do convolution on the image and through a series of cascaded convolutions -and applying non linear function to each result- we may get the NN to learn the weignts of each layer's filters that will enable us to classiy the image or do the required task 

- so , what we want here is to learn the weights  that do the required mapping between the inputs and the label 

# Introduction to Convolutional Neural Networks

This lB addresses the basic concepts of Convolutional Neural Networks and their implementation using the Keras framework.
Convolutional Neural Networks (CNNs) are a class of feed-forward artificial neural networks that are applied to analyze visual 2D imagery, meaning that we can feed images directly to a CNN without the need to flatten them into a 1D vector beforehand.
CNNs have revolutionized the field of computer vision in the last decade. In 2012 Alex Krizhevsky introduced the AlexNet architecture to win the ImageNet Challenge (one of the most important competitions on image classification within the Computer Vision community), by reducing the top-5 error more than 10 percentage points, which was an incredible improvement. As of now, CNNs are used not only on image classification but in many other computer vision tasks.



# CNN Structure

The basic pipeline of CNNs consists of an image that undergoes through a series of convolutional layers to obtain a final representation. This final representation depends on the kind of problem that the architecture is facing. For instance, the output of the final layer in a classification setup will be the probabilities of each of the classes. However, the network's outputs take any shape that is desired; we could use a single value for regression or a newly generated image for semantic segmentation or image transformation purposes. Layers within CNNs are similar to those that we have already seen on 1D neural networks, but having this time 2D matrices as inputs instead of 1D vectors. Now, we will introduce specific layers that are used in CNNs. 

# 2D Convolutional Layer

The most common layer in any CNN architecture is the 2D convolutional layer. Convolutional layers are specifically designed to extract features from images or even extract features from previously extracted features. Therefore, in CNN networks, we can stack several convolutional layers to obtain more and more complex representations -features- of an image.

![](https://cdn-images-1.medium.com/max/800/1*Fw-ehcNBR9byHtho-Rxbtw.gif)

Image [source](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1).

The latest deep learning frameworks have made possible the integration of convolutional layers easily on our architectures with only a single line of code. We will address here how 2D convolutions work since full understanding is needed to comprehend how any CNN operates. The following images and some explanations can also be found on [Irhum Shafkat's blog](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) and [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/) book. Both are strongly recommended.

To understand 2D convolutions, we need to define first what a kernel is. Kernels are simply matrices of numbers. The numbers on the kernels are the so-called weights and the weights on the kernels change as we train the network. Training the network, and therefore updating the kernel's weights, will improve the performance of CNN in solving specific tasks. 

The 2D convolution operation takes the network's kernels and follows a process similar to a sliding window over the input image (or feature map), performing an element-wise multiplication with the elements that are currently on. See the figure below. The results of this elementwise multiplication are summed up, giving, as a result, a single output value. This operation is repeated for all the positions of this sliding window, composing at the end the feature map. This generated feature map can go through another 2D convolutional layer and create more powerful features. 

> 

![](https://cdn-images-1.medium.com/max/800/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

> 
The previous image shows the 2D convolution operation. A new feature value is the weighted sum of all the elements in the sliding window after the elementwise multiplication between input and kernel.  The bigger the size of the kernel is, the more feature elements will contribute to the final output value. In contrast to fully connected layers, where a new feature value is a weighted sum over all input values, 2D convolutions compute features based on local areas. In other words, instead of looking at every input component, they consider only features coming from close locations. 

In the above example, the input image on the left has a size of 5x5 and the dimension of the resulting feature map is 3x3, showing that the size of the output maps is not always equal to the input size. The output size can be computed by doing:

$O = W - K + 1$,

where $O$ is the output height/length, $W$ is the input height/length and $K$ is the kernel size. The output size is not only conditioned on the input size but also on the kernel size. Check in the following code cell how the output feature map shape changes as you increase the kernel size. In Keras, we define the layer by using `Conv2D` from `keras.layers` (documentation [here](https://keras.io/layers/convolutional/#conv2d)).









In [1]:
output_width = 100 - 3 + 1
output_height= 100 - 3 + 1
output_width, output_height

(98, 98)

- in today's lab, we will do some convolutional layers, make sure of running them then study some architectures 

in keras the layer is 

`Conv2D(number of filters, kernel size, strides = 1,padding = 'valid')`

In [2]:
import numpy as np
import keras
import tensorflow as tf
#tf.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from keras.models import Sequential
from keras.layers import Conv2D

# Generate dummy data -image-  
input_feature = np.random.random((1, 100, 100, 1)) # a patch of 1 image, each of size 100x100x1

# input: 100x100 image with 1 channels -> (100, 100, 1) tensor.
# this applies 1 convolution filter of size 3x3 each.
model = Sequential()
# conv2D(number of filters, kernel size, stride = 1)
model.add(Conv2D(1, (3, 3), input_shape=(100, 100, 1))) # we don't write the patch in dimension 


output_feature = model.predict(input_feature)

print('Input size: ({:}, {:})'.format(input_feature.shape[1], input_feature.shape[2]))
print('Output size: ({:}, {:})'.format(output_feature.shape[1], output_feature.shape[2]))

1/1 [==============================] - 0s 440ms/step
Input size: (100, 100)
Output size: (98, 98)


In [3]:
input_feature.shape,output_feature.shape

((1, 100, 100, 1), (1, 98, 98, 1))

Moreover, the kernel size and the input size are not the only parameters affecting the output size. Two extra elements that change the size of the output map are padding and striding.

# Adding Padding to Input Features

In some tasks, such as image translation, we need the output size to be equal to the input size. The solution to that is using padding, where extra edges are added to the input features so that the dimension is not reduced after the convolutional layer. Normally those pixels have $0$ value (termed zero-padding), but depending on the application other methods could be used, e.g., reflection or symmetric padding.

![](https://cdn-images-1.medium.com/max/800/1*1okwhewf5KCtIPaFib4XaA.gif)

If padding is used, the new output size can be computed by doing:

$O = W - K + 2P+ 1$, 

where $P$ is the padding value. $P$ must be set in concordance with the kernel size if dimensionality wants to be preserved. Padding can be added to the `Conv2d` layer in Keras by using the padding argument and selecting one of the two settings: `valid`, which is the default value and means no padding; and `same`, which adapts the padding value to have the same output size as input size.

In [4]:
import numpy as np
import keras
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential
from keras.layers import Conv2D

# Generate dummy data
input_feature = np.random.random((1, 100, 100, 1))

# input: 100x100 image with 1 channels -> (100, 100, 1) tensor.
# this applies 1 convolution filter of size 3x3 each.
# attribute padding='same' applies zero-padding to the input feature map
model = Sequential()
model.add(Conv2D(1, (3, 3), input_shape=(100, 100, 1), padding="same")) # this will adjust the padding frames according to the kernel size, therefore roduces output of the SAME size

output_feature = model.predict(input_feature)

print('Input size: ({:}, {:})'.format(input_feature.shape[1], input_feature.shape[2]))
print('Output size: ({:}, {:})'.format(output_feature.shape[1], output_feature.shape[2]))

1/1 [==============================] - 0s 77ms/step
Input size: (100, 100)
Output size: (100, 100)


In [5]:
output_width = 100 - 3 + 2 + 1
output_height = 100 - 3 + 2 + 1
output_width, output_height

(100, 100)

# Using Stride in Convolutional Layers

The stride operation allows the convolutional layers to skip some of the sliding windows explained above. Therefore, instead of jumping one pixel apart, we can define the number of skipped elements before computing the weighting sum between the kernel's weights and input features. A stride of 1 means that features will be extracted from all windows a pixel apart, so basically, every single window will be computed, acting as a standard convolution. A stride of 2 means that we are selecting windows 2 pixels apart, skipping every other window in the process. Strides reduce the number of computations and consequently the size of the output map. In practice, as we go deeper into the CNN, the size of the feature map gets smaller while the number of channels increases. Moreover, we can further reduce the size of the feature map using pooling operations, which we introduce later in this tutorial.

![](https://cdn-images-1.medium.com/max/800/1*BMngs93_rm2_BpJFH2mS0Q.gif)

If strides are used, the new output size can be computed as:

$O = \dfrac{W - K + 2P}{S}+ 1$, 

where $S$ is the stride value. The stride can be set in the layer by using the `strides` argument.

In [6]:
import numpy as np
import keras
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential
from keras.layers import Conv2D

# Generate dummy data
input_feature = np.random.random((1, 100, 100, 1))

# input: 100x100 image with 1 channels -> (100, 100, 1) tensor.
# this applies 1 convolution filter of size 3x3 each.
# attribute padding='same' applies zero-padding to the input feature map
# attribute strides=2 applies applies stride of 2
model = Sequential()
model.add(Conv2D(1, (3, 3), input_shape=(100, 100, 1), padding="same", strides=2))

output_feature = model.predict(input_feature)

print('Input size: ({:}, {:})'.format(input_feature.shape[1], input_feature.shape[2]))
print('Output size: ({:}, {:})'.format(output_feature.shape[1], output_feature.shape[2]))

1/1 [==============================] - 0s 67ms/step
Input size: (100, 100)
Output size: (50, 50)


In [7]:
output_width = ((100 - 3 + 2) / 2) + 1
output_width

50.5

# Differences Between Kernel and Filters

Filter = kernel x no. of channels

The examples above take as input a single-channel image and compute a feature map with also one channel. However, when dealing with RGB images or feature maps, the input is no longer a single-channel map but, instead, they can have multiple channels. In the case of an RGB image, for each 2D convolution, we will need to define 3 kernels to interact with each of the image's channel colors. This group of kernels is called a filter. Thus, a filter is a collection of kernels that produces a single output.

As a regular practice when defining Deep Learning models, we increase the number of filters in each convolutional layer in order to obtain more channels from those layers that could extract more complex and meaningful features. When designing a neural network, each filter in a convolutional layer must have the same number of kernels as the number of channels of the input feature. Keras already deals with the number of kernels inside each filter by keeping track of the input size in each convolutional layer, unlike other frameworks such as Pytorch or TensorFlow. Hence, in Keras, we must only decide the number of filters (output channels) in each layer.

The next figure shows how the convolution is performed when having three input channels. First, one filter uses its three independent kernels to convolve with the RGB channels of the input image:

![](https://cdn-images-1.medium.com/max/1000/1*8dx6nxpUh2JqvYWPadTwMQ.gif)

Next, each of the processed feature maps is summed together to obtain a single channel:

![](https://cdn-images-1.medium.com/max/1000/1*CYB2dyR3EhFs1xNLK8ewiA.gif)

Finally, we add the bias term to obtain the feature map. There is a single bias, therefore the bias gets added to the full output channel to produce a single-channel feature map. This operation is repeated for all filters on its own since each filter has a different set of kernels and a scalar bias, combining them at the end and building the final feature map. 

Now, we show how to use a `Conv2D` layer that takes an input image with 3 channels and generates an output map with 32 channels.






- let's try Conv2D with input of thickness > 1
    - we will input RGB image (input of 3 channels)
    - we want to output an image with 32 channel -which means we will use 32 filters- each filter will adjust its thickness to match the input thickness 

In [8]:
import numpy as np
import keras
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential
from keras.layers import Conv2D

# Generate dummy data
input_feature = np.random.random((1, 100, 100, 3))

# input: 100x100 image with 3 channels -> (100, 100, 3) tensor.
# this applies 32 convolution filters of size 3x3 each.
# attribute padding='same' applies zero-padding to the input feature map
# attribute strides=2 applies applies stride of 2
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3), padding="same", strides=2))

output_feature = model.predict(input_feature)

print('Input size: ({:}, {:}, {:})'.format(input_feature.shape[1], input_feature.shape[2], input_feature.shape[3]))
print('Output size: ({:}, {:}, {:})'.format(output_feature.shape[1], output_feature.shape[2], output_feature.shape[3]))

1/1 [==============================] - 0s 123ms/step
Input size: (100, 100, 3)
Output size: (50, 50, 32)


- we go 50 because we used strides = 2 and we got 50 and not 49 because we used padding to be same and we got 32 channels in the output as we used 32 filters 

# Activation Functions

- so far we have done only convolution on the layers (which is not what we do in the CNN) so we ALWAYS apply an activation function after we convolute  



As seen in previous LABS, after `Dense` layers usually we can find activation functions, which we will also use after `Conv2D` layers. Those activation functions are a set of operators that will map the feature values to a new set of resulting values, depending on the function at hand. The main reason for using activation functions is that they add non-linearities to the network, giving more expressive power to the network, which will be able to reproduce more complex functions. 


The complete list of activation functions that Keras offers can be found [here](https://keras.io/activations/). We introduce here how some of them:

*  **Sigmoid Function** sets the output in the range (0, 1). The sigmoid function is widely used in binary classification problems since its output can be taken as a probability value. `keras.activations.sigmoid(x)`:

>![](https://i.ibb.co/Ph8dsTv/sigmoid.png)

*  **Tanh Function** is a logistic function as sigmoid, but the range of the tanh function is (-1, 1). Contrary to sigmoid function, where the values close to 0 are set around 0.5, in the tanh function they will be still mapped around the 0 value. `keras.activations.tanh(x)`:

>![](https://i.ibb.co/68g7LpL/tanh.png)

*  **ReLU Function** is the most common activation function you can find in any current CNN as in general works better than the rest. The range of this function in \[0, inf). Basically, it sets all negative values to 0, and hence is computationally easy to implement. As a drawback, during training some neurons will *die*, meaning that the output will be 0 for all available data points and no gradient will be propagated there. `keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)`:

>![](https://i.ibb.co/Zd9H8Z4/relu.png)

*  **LeakyReLU Function** is a modified version of the ReLU activation above, which attempts to solve the problem of dying neurons that ReLU has. While ReLU does not backpropagate negative values, Leaky ReLU smooths those values without setting them to 0. That allows the gradients to backpropagate through the network even for negative values. `keras.layers.LeakyReLU(alpha=0.3)`:

>![](https://i.ibb.co/dmnJ6h1/leakyrelu.png)

*  **Softmax Function** is another widely activation function for multi-class classification problems and usually is employed as the last activation function in a multi-class classification model. This function sets all of the output elements to the range (0, 1). However, the softmax function does not take independently the input values to map in their probability values, it takes an un-normalized vector, $s$, and normalizes it into a probability distribution, $p$. As the output is a probability, the output elements add up to 1. `keras.activations.softmax(x, axis=-1)`. The output value $p_i$ is computed as:

> $p_{i} = \dfrac{e^{s_i}}{\sum_{\substack{j}}^{N} e^{s_j}}$





The following example shows the feature maps before and after of the ReLU activation function. All values that are negative are set to 0 after the activation function as explained above. 

tip: to get output of a certain layer

`from keras.models import Model`
then give the layer a name then 

`sliced_model = Model(inputs = original_model.input , outputs = original_model.get_layer(layer_name).output )`

In [9]:
import numpy as np
import keras
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential, Model
from keras.layers import Conv2D, Activation

# Generate dummy data
input_feature = np.random.random((1, 3, 3, 1)) - 0.5
 
# input: 3x3 image with 1 channel -> (3, 3, 1) tensor.
# This applies a 1 convolution filter of size 3x3 each.
# This applies a ReLU activation function
model = Sequential()
model.add(Conv2D(1, (3, 3), input_shape=(3, 3, 1), padding="same", name="conv"))
model.add(Activation("relu")) # we added activation as a separate layer, but we could have put it in the conv2D instead

model_before_ReLU = Model(inputs=model.input, outputs=model.get_layer("conv").output)

output_feature = model_before_ReLU.predict(input_feature)
output_ReLu_feature = model.predict(input_feature)

print('Output Network without activation function')
print(output_feature)

print('')
print('Output Network after ReLU activation function')
print(output_ReLu_feature)


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 119ms/step
Output Network without activation function
[[[[ 0.3135101 ]
   [ 0.43296742]
   [ 0.2926268 ]]

  [[ 0.1481274 ]
   [ 0.2183942 ]
   [ 0.11018097]]

  [[-0.10441318]
   [-0.14919026]
   [-0.2297165 ]]]]

Output Network after ReLU activation function
[[[[0.3135101 ]
   [0.43296742]
   [0.2926268 ]]

  [[0.1481274 ]
   [0.2183942 ]
   [0.11018097]]

  [[0.        ]
   [0.        ]
   [0.        ]]]]


- we notice that the model before applying the acivation has a +ve and -ve parts 
- but after applying the relu we have either +ve or 0

# Pooling Layer

It is a common practice to insert a pooling layer between convolutional layers in CNNs. In a standard CNN architecture, we set the feature sizes to become smaller progressively to reduce the number of parameters and the computation in the networks, and also to merge the information from different spatial locations. To reduce the feature map sizes, we can either use bigger strides in the convolutional layers or we can use pooling layers. Pooling layers perform a spatial sliding window and apply an operation to reduce the spatial size. Those operations vary depending on the architecture, being the max, mean and min pooling the most typical ones. Here, we will explain the max pooling, although all the others work similarly. Max pooling keeps only the max value in a neighborhood, where the neighborhood is defined by the size of the kernel. The next example shows the result of a Max Pooling layer with a 2x2 kernel and a stride of 2.

![](https://i.ibb.co/Xp454S4/MaxPool.png)

As in convolutional layers, the final size is conditioned to the stride size of the pooling layer. However, contrary to convolutional layers, pooling layers operate independently on each of the input channels, without modifying the depth of the feature maps. To add max pooling to our model we need to import `MaxPooling2d` from `keras.layers` and define the strides and the pooling size.

- `maxpooling2D(pool_size=(,) , strides = (,) )` notice that the pooling size on x does not have to be the same size on y
    - example poolsize can be equal to (2,3) (this will divide the image to 2x3 pools
    - if we wrote a single number 2 it will be casted internally to be 2x2
    
tip: the strides as well can be treated like that 
- strides 2 is internally casted to 2x2 (will skip 2 pixels to the right and 2 pixels down 
- strides (m,n) will skip m pixels to the right and b pixels down

i guess the **max pooling output size** also can be estimated using the formula we used with the **conv2D layer output size** 

In [10]:
import numpy as np
import keras
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D

# Generate dummy data
input_feature = np.random.random((1, 100, 100, 3))

# input: 100x100 image with 3 channels -> (100, 100, 3) tensor.
# this applies 32 convolution filters of size 3x3 each.
# attribute padding='same' applies zero-padding to the input feature map
# attribute strides=1 applies applies stride of 1
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3), padding="same", strides=1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))



output_feature = model.predict(input_feature)

print('Input size: ({:}, {:}, {:})'.format(input_feature.shape[1], input_feature.shape[2], input_feature.shape[3]))
print('Output size: ({:}, {:}, {:})'.format(output_feature.shape[1], output_feature.shape[2], output_feature.shape[3]))

1/1 [==============================] - 0s 84ms/step
Input size: (100, 100, 3)
Output size: (50, 50, 32)


# Classification on MNIST

in this section will be seeing how to perform image classification when the input data is a 2D image instead of a flat 1D vector. 

As discussed above, Convolutional Neural Networks aim to extract and exploit the local relationships on 2D maps, thus, CNNs are much more convenient for images than Multi-layer Perceptron models. 

First of all, we load the MNIST dataset from Keras' framework. The definition of the data is almost identical than in the Keras tutorial, although this time we are not reshaping the input images to have a single dimension.

In [11]:
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.utils import np_utils
from keras.datasets import mnist
from tensorflow.keras.optimizers import RMSprop
#from tensorflow.keras.optimizers import rmsprop
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras import regularizers
import numpy as np

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("xtrain before expansion: " + str(x_train.shape))

shape = x_train.shape

# Normalize and reshape the input images, we expand the shape to contain the axis for the number of channels (so that we have series of 3D images)
x_train = np.expand_dims(x_train.astype('float32'), -1) # -1 means that the new axis will be added to the last
x_test = np.expand_dims(x_test.astype('float32'), -1)

# normalize the values to be from 0 to 1
x_train /= 255
x_test /= 255

# convert the labels to one hot vectors  
y_train_class = np_utils.to_categorical(y_train, 10)
y_test_class = np_utils.to_categorical(y_test, 10)

print('Image shape: {0}'.format(x_train.shape[1:])) #print the single image shape
print('Total number of training samples: {0}'.format(x_train.shape[0]))
print('Total number of test samples: {0}'.format(x_test.shape[0]))

print("xtrain after expansion: " + str(x_train.shape))

11490434/11490434 [==============================] - 0s 0us/step
xtrain before expansion: (60000, 28, 28)
Image shape: (28, 28, 1)
Total number of training samples: 60000
Total number of test samples: 10000
xtrain after expansion: (60000, 28, 28, 1)


In [12]:
dummy = np.random.random(size=(28,28,1))
dummy.shape

(28, 28, 1)

In [13]:
np.expand_dims(dummy,0).shape

(1, 28, 28, 1)

In [14]:
np.expand_dims(dummy,1).shape

(28, 1, 28, 1)

In [15]:
np.expand_dims(dummy,2).shape

(28, 28, 1, 1)

In [16]:
np.expand_dims(x_train,-1).shape

(60000, 28, 28, 1, 1)

In [17]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [18]:
np_utils.to_categorical(y_train,num_classes=10)

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

As illustrated in the cell, this time the input image before the model is 28x28x1. 

Now we can define a model composed of convolutional layers, activation functions, and maxpool operators:

In [19]:
# Declare Convolutional Part
model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [20]:
model = Sequential([
    Conv2D(16,(3,3),padding='same',input_shape=(x_train.shape[1:]),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(16,(3,3),padding='same',activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    # after the CNN layers, we need to flatten the resulting output 
    tf.keras.layers.Flatten(),
    # now we input to a dense layer (which will be the final output layer)
    Dense(10,activation='softmax')
    
])

As explained in the previous labs, in a classification problem, the output of the model is a vector containing the probabilities of the input image of belonging to a specific class. 

Now the resulting feature map of the model is a map with the shape *Batch x Weight' x Height' x Channel*, and needs to be mapped to a vector with shape *Batch x Num Classes*. A common technique to process this mapping is to add a Flatten layer that will reshape the feature map to *Batch x (Weight' * Height' * Channel)*, and next to add a dense layer which brings this new feature map to the output desired size. 

In [21]:
# Declare FCN Part
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("softmax"))

Let's visualize the model's shape and outputs in each layer.

In [22]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 10)               

Now we can train our CNN and check its performance on MNIST. 

In [23]:
# initiate RMSprop optimizer
opt = RMSprop(lr=0.0003, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train_class, batch_size=1024, epochs=10,  verbose=1)

score = model.evaluate(x_test, y_test_class, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
59/59 [==============================] - 37s 585ms/step - loss: 2.2666 - accuracy: 0.2424
Epoch 2/10
22/59 [==========>...................] - ETA: 19s - loss: 2.1806 - accuracy: 0.3900

KeyboardInterrupt: ignored

- why does a cnn layer contain biases? (the number of biases = number of filters, why?)